# Houses task solver

In [12]:
import numpy as np
import copy
import re

In [66]:
def load_task(filename):
    with open('task1.txt', 'r') as f:
        contents =  f.readlines()
        f.close()
    
    contents = [re.findall('[0-9]+', contents[i])[0] for i in range(len(contents))]
    row = []
    plane = []
    col = [int(char) for char in contents[0]]
    for i in range(1, len(contents)):
        row.append(int(contents[i][0]))
        plane.append([int(contents[i][j]) for j in range(1, len(contents[i]))])
    plane = np.array(plane)
    
    return row, col, plane

def print_task(row, col, plane):
    text = '  ' + ' '.join(str(x) for x in col) + '\n  ' + '_' * (2 * len(col) - 1) + '\n'
    for i in range(len(row)):
        text += str(row[i]) + '|' + ' '.join(str(x) for x  in plane[i]) + '\n'
    print(text)

In [81]:
row, col, game_plane = load_task('task1.txt')
print_task(row, col, game_plane)

  1 1 2 1 1 1
  ___________
1|0 1 0 0 0 0
0|0 0 0 0 0 0
2|0 0 0 0 0 0
1|0 0 1 0 1 0
2|1 0 0 0 1 0
1|0 0 1 0 0 1



In [2]:
row = [1, 0, 2, 1, 2, 1]
col = [1, 1, 2, 1, 1, 1]


game_plane = [[0, 1, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0],
              [0, 0, 1, 0, 1, 0],
              [1, 0, 0, 0, 1, 0],
              [0, 0, 1, 0, 0, 1]]
game_plane = np.array(game_plane)

In [78]:
row = [5, 1, 4, 1, 4, 2, 4, 2, 1, 4]
col = [3, 2, 0, 5, 0, 5, 0, 2, 2, 3, 0, 2, 2, 2]

game_plane = [[0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0],
              [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0],
              [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
              [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
              [0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]]
game_plane = np.array(game_plane)

In [70]:
check_idxs = [[i, j] for i in range(-1, 2) for j in range(-1, 2) if i != 0 or j != 0]
check_idxs

[[-1, -1], [-1, 0], [-1, 1], [0, -1], [0, 1], [1, -1], [1, 0], [1, 1]]

In [71]:
insert_idxs = [[-1, 0], [1, 0], [0, -1], [0, 1]]

In [82]:
x, y = np.where(game_plane == 1)
houses = [[x[i], y[i]] for i in range(len(x))]
houses

[[0, 1], [3, 2], [3, 4], [4, 0], [4, 4], [5, 2], [5, 5]]

In [76]:
def in_plane(i, j, plane):
    if i < 0 or i >= len(plane) or j < 0 or j >= len(plane[0]):
        return False
    return True

def check_square(i, j, plane):
    if plane[i][j] > 0:
        return False
    if np.sum(plane[i,:] > 1) < row[i] and np.sum(plane[:,j] > 1) < col[j]:
        for idx in check_idxs:
            x = idx[0] + i
            y = idx[1] + j
            if in_plane(x, y, plane):
                if plane[x][y] > 1:
                    return False
        return True
    return False

In [74]:
def solve(plane, h):
    if h == len(houses):
        print_task(row, col, plane)
        return
    inserted = False
    for idx in insert_idxs:
        x = idx[0] + houses[h][0]
        y = idx[1] + houses[h][1]
        if in_plane(x, y, plane):
            if check_square(x, y, plane):
                inserted = True
                plane_copy = copy.deepcopy(plane)
                plane_copy[x][y] = 2
                solve(plane_copy, h+1)
                
    if not inserted:
        return
        

In [83]:
solve(game_plane, 0)

  1 1 2 1 1 1
  ___________
1|0 1 2 0 0 0
0|0 0 0 0 0 0
2|0 0 2 0 2 0
1|2 0 1 0 1 0
2|1 0 0 2 1 2
1|0 2 1 0 0 1

